In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'

## 한강
## 자동
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
    


In [7]:
target_col = '총유기탄소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:11', ':,28:45', ':,26:29']
nakdong = [':,2:11', ':,2:12']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,2:11'

In [8]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [9]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [10]:
def normalize(df):
    # normalize data
    
    
    df_all = pd.concat(df)
    
#     train_mean = df_all.mean()
#     train_std = df_all.std()
#     for i in range(len(file_names)):
#         df[i] = (df[i]-train_mean)/train_std

    train_mean = df_all.min()
    train_std = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/(train_std - train_mean)
    
    return df_all, train_mean, train_std

In [11]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)

가평_2016.xlsx
(8784, 12)
(8784, 12)
가평_2017.xlsx
(8744, 12)
(8760, 12)
가평_2018.xlsx
(8730, 12)
(8760, 12)
가평_2019.xlsx
(8760, 12)
(8760, 12)
의암호_2016.xlsx
(8784, 11)
(8784, 11)
의암호_2017.xlsx
(8760, 11)
(8760, 11)
의암호_2018.xlsx
(8746, 11)
(8760, 11)
의암호_2019.xlsx
(8760, 11)
(8760, 11)
서상_2016.xlsx
(8784, 11)
(8784, 11)
서상_2017.xlsx
(8760, 11)
(8760, 11)
서상_2018.xlsx
(8751, 11)
(8760, 11)
서상_2019.xlsx
(8760, 11)
(8760, 11)


In [12]:
train_mean, train_std

(수온           0.300
 수소이온농도       6.200
 전기전도도       48.000
 용존산소         2.900
 탁도           0.000
 총유기탄소        0.700
 총질소          0.346
 총인           0.000
 클로로필-a       0.000
 Day sin     -1.000
 Day cos     -1.000
 Year sin    -1.000
 Year cos    -1.000
 dtype: float64,
 수온           32.400
 수소이온농도       10.000
 전기전도도       191.000
 용존산소         16.400
 탁도          290.300
 총유기탄소         7.200
 총질소           5.324
 총인            0.395
 클로로필-a      159.100
 Day sin       1.000
 Day cos       1.000
 Year sin      1.000
 Year cos      1.000
 dtype: float64)

In [13]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,0.127726,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.017599,0.500000,1.000000,0.498667,0.999998
1,0.121495,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.016970,0.629410,0.982963,0.499025,0.999999
2,0.112150,0.421053,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.016970,0.750000,0.933013,0.499384,1.000000
3,0.102804,0.394737,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.017599,0.853553,0.853553,0.499742,1.000000
4,0.096573,0.394737,0.405594,0.711111,0.002067,0.092308,NaN,NaN,0.016342,0.933013,0.750000,0.500100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.115265,0.315789,0.440559,0.688889,0.001378,0.138462,0.248694,0.022785,0.086109,0.017037,0.629410,0.497133,0.999992
35060,0.099688,0.315789,0.447552,0.696296,0.001378,0.138462,0.231619,0.022785,0.082338,0.066987,0.750000,0.497491,0.999994
35061,0.087227,0.315789,0.447552,0.703704,0.001378,0.153846,0.249699,0.022785,0.079824,0.146447,0.853553,0.497850,0.999995
35062,0.077882,0.315789,0.454545,0.711111,0.001378,0.138462,0.229409,0.025316,0.079195,0.250000,0.933013,0.498208,0.999997


In [14]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,0.127726,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.017599,0.500000,1.000000,0.498667,0.999998
1,0.121495,0.421053,0.405594,0.725926,0.002067,0.092308,NaN,NaN,0.016970,0.629410,0.982963,0.499025,0.999999
2,0.112150,0.421053,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.016970,0.750000,0.933013,0.499384,1.000000
3,0.102804,0.394737,0.412587,0.711111,0.002067,0.092308,NaN,NaN,0.017599,0.853553,0.853553,0.499742,1.000000
4,0.096573,0.394737,0.405594,0.711111,0.002067,0.092308,NaN,NaN,0.016342,0.933013,0.750000,0.500100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.115265,0.315789,0.440559,0.688889,0.001378,0.138462,0.248694,0.022785,0.086109,0.017037,0.629410,0.497133,0.999992
35060,0.099688,0.315789,0.447552,0.696296,0.001378,0.138462,0.231619,0.022785,0.082338,0.066987,0.750000,0.497491,0.999994
35061,0.087227,0.315789,0.447552,0.703704,0.001378,0.153846,0.249699,0.022785,0.079824,0.146447,0.853553,0.497850,0.999995
35062,0.077882,0.315789,0.454545,0.711111,0.001378,0.138462,0.229409,0.025316,0.079195,0.250000,0.933013,0.498208,0.999997


In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    22     3     0]
 [    3     9     1    22]
 [    4    14     2    31]
 ...
 [ 5451     9     9 19454]
 [ 5460     3     3 19463]
 [ 5463     7     1 19466]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,4.0,7.4,111.0,12.2,0.4,1.6,1.584,0.009,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.4,112.0,12.3,0.4,1.6,1.499,0.009,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.1,7.4,112.0,12.4,0.4,1.7,1.589,0.009,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35062,2.8,7.4,113.0,12.5,0.4,1.6,1.488,0.010,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 13])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  9335


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 37.1562 - disc_loss: 0.7069 - rmse: 0.5223 - val_loss: 0.8044
Epoch 2/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 71.7968 - disc_loss: 0.5170 - rmse: 0.8043 - val_loss: 0.3482
Epoch 3/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 17.2287 - disc_loss: 0.4711 - rmse: 0.3500 - val_loss: 0.3058
Epoch 4/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 14.4643 - disc_loss: 0.4060 - rmse: 0.3101 - val_loss: 0.2665
Epoch 5/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 10.9964 - disc_loss: 0.4027 - rmse: 0.2627 - val_loss: 0.2190
Epoch 6/2000
1/1 [==============================] - 0s 161ms/step - gen_loss: 8.6262 - disc_loss: 0.3891 - rmse: 0.2192 - val_loss: 0.1922
Epoch 7/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 7.2495 - disc_loss: 0.3939 - rmse: 0.1923 - val_loss: 0.1773
Epoch

1/1 [==============================] - 0s 140ms/step - gen_loss: 0.5521 - disc_loss: 0.2977 - rmse: 0.0738 - val_loss: 0.0821
Epoch 60/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 0.5549 - disc_loss: 0.2808 - rmse: 0.0738 - val_loss: 0.0717
Epoch 61/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.5627 - disc_loss: 0.2917 - rmse: 0.0698 - val_loss: 0.0689
Epoch 62/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.5409 - disc_loss: 0.2770 - rmse: 0.0727 - val_loss: 0.0658
Epoch 63/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 0.5096 - disc_loss: 0.2772 - rmse: 0.0698 - val_loss: 0.0781
Epoch 64/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 0.5441 - disc_loss: 0.2891 - rmse: 0.0719 - val_loss: 0.0761
Epoch 65/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 0.5807 - disc_loss: 0.2906 - rmse: 0.0757 - val_loss: 0.0730
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 141ms/step - gen_loss: 0.4844 - disc_loss: 0.2379 - rmse: 0.0659 - val_loss: 0.0582
Epoch 118/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4269 - disc_loss: 0.2319 - rmse: 0.0620 - val_loss: 0.0662
Epoch 119/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 0.4303 - disc_loss: 0.2308 - rmse: 0.0628 - val_loss: 0.0652
Epoch 120/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 0.4830 - disc_loss: 0.2255 - rmse: 0.0658 - val_loss: 0.0638
Epoch 121/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 0.4355 - disc_loss: 0.2275 - rmse: 0.0626 - val_loss: 0.0625
Epoch 122/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 0.4593 - disc_loss: 0.2313 - rmse: 0.0608 - val_loss: 0.0655
Epoch 123/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 0.4118 - disc_loss: 0.2296 - rmse: 0.0596 - val_loss: 0.0585
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 133ms/step - gen_loss: 0.4671 - disc_loss: 0.2138 - rmse: 0.0577 - val_loss: 0.0611
Epoch 176/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 0.4694 - disc_loss: 0.1994 - rmse: 0.0553 - val_loss: 0.0634
Epoch 177/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.4980 - disc_loss: 0.1988 - rmse: 0.0614 - val_loss: 0.0583
Epoch 178/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.5006 - disc_loss: 0.2027 - rmse: 0.0612 - val_loss: 0.0574
Epoch 179/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 0.5202 - disc_loss: 0.2048 - rmse: 0.0599 - val_loss: 0.0581
Epoch 180/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 0.4648 - disc_loss: 0.1994 - rmse: 0.0595 - val_loss: 0.0586
Epoch 181/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 0.4827 - disc_loss: 0.1991 - rmse: 0.0589 - val_loss: 0.0606
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 151ms/step - gen_loss: 0.4821 - disc_loss: 0.1904 - rmse: 0.0509 - val_loss: 0.0565
Epoch 234/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 0.4488 - disc_loss: 0.1937 - rmse: 0.0500 - val_loss: 0.0499
Epoch 235/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.4541 - disc_loss: 0.1862 - rmse: 0.0513 - val_loss: 0.0523
Epoch 236/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 0.4605 - disc_loss: 0.1843 - rmse: 0.0488 - val_loss: 0.0470
Epoch 237/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 0.4582 - disc_loss: 0.1848 - rmse: 0.0525 - val_loss: 0.0527
Epoch 238/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 0.4753 - disc_loss: 0.1955 - rmse: 0.0530 - val_loss: 0.0542
Epoch 239/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.4688 - disc_loss: 0.1924 - rmse: 0.0526 - val_loss: 0.0470
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 151ms/step - gen_loss: 0.6452 - disc_loss: 0.1216 - rmse: 0.0361 - val_loss: 0.0337
Epoch 1274/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 0.5881 - disc_loss: 0.1224 - rmse: 0.0339 - val_loss: 0.0349
Epoch 1275/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.6043 - disc_loss: 0.1241 - rmse: 0.0354 - val_loss: 0.0341
Epoch 1276/2000
1/1 [==============================] - 0s 159ms/step - gen_loss: 0.5490 - disc_loss: 0.1208 - rmse: 0.0327 - val_loss: 0.0295
Epoch 1277/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 0.6384 - disc_loss: 0.1229 - rmse: 0.0329 - val_loss: 0.0325
Epoch 1278/2000
1/1 [==============================] - 0s 167ms/step - gen_loss: 0.5673 - disc_loss: 0.1201 - rmse: 0.0321 - val_loss: 0.0329
Epoch 1279/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 0.6121 - disc_loss: 0.1261 - rmse: 0.0330 - val_loss: 0.0360
Epoch 1280/2000
1/1 [=

1/1 [==============================] - 0s 149ms/step - gen_loss: 0.5119 - disc_loss: 0.1197 - rmse: 0.0357 - val_loss: 0.0318
Epoch 1331/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 0.4846 - disc_loss: 0.1150 - rmse: 0.0311 - val_loss: 0.0317
Epoch 1332/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.5325 - disc_loss: 0.1179 - rmse: 0.0348 - val_loss: 0.0314
Epoch 1333/2000
1/1 [==============================] - 0s 162ms/step - gen_loss: 0.5280 - disc_loss: 0.1180 - rmse: 0.0345 - val_loss: 0.0317
Epoch 1334/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.4740 - disc_loss: 0.1175 - rmse: 0.0330 - val_loss: 0.0305
Epoch 1335/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 0.5020 - disc_loss: 0.1197 - rmse: 0.0300 - val_loss: 0.0350
Epoch 1336/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 0.5510 - disc_loss: 0.1199 - rmse: 0.0338 - val_loss: 0.0335
Epoch 1337/2000
1/1 [=

1/1 [==============================] - 0s 147ms/step - gen_loss: 0.5924 - disc_loss: 0.1158 - rmse: 0.0343 - val_loss: 0.0311
Epoch 1388/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 0.5624 - disc_loss: 0.1227 - rmse: 0.0374 - val_loss: 0.0359
Epoch 1389/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 0.5696 - disc_loss: 0.1185 - rmse: 0.0376 - val_loss: 0.0335
Epoch 1390/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 0.5651 - disc_loss: 0.1176 - rmse: 0.0338 - val_loss: 0.0353
Epoch 1391/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 0.5405 - disc_loss: 0.1185 - rmse: 0.0400 - val_loss: 0.0340
Epoch 1392/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 0.5798 - disc_loss: 0.1213 - rmse: 0.0354 - val_loss: 0.0317
Epoch 1393/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 0.4964 - disc_loss: 0.1205 - rmse: 0.0337 - val_loss: 0.0382
Epoch 1394/2000
1/1 [=

1/1 [==============================] - 0s 134ms/step - gen_loss: 0.5173 - disc_loss: 0.1205 - rmse: 0.0337 - val_loss: 0.0369
Epoch 1445/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 0.5985 - disc_loss: 0.1191 - rmse: 0.0325 - val_loss: 0.0364
Epoch 1446/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 0.5785 - disc_loss: 0.1176 - rmse: 0.0377 - val_loss: 0.0290
Epoch 1447/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 0.5762 - disc_loss: 0.1164 - rmse: 0.0339 - val_loss: 0.0320
Epoch 1448/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.5754 - disc_loss: 0.1200 - rmse: 0.0398 - val_loss: 0.0317
Epoch 1449/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 0.5685 - disc_loss: 0.1199 - rmse: 0.0328 - val_loss: 0.0322
Epoch 1450/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.5432 - disc_loss: 0.1194 - rmse: 0.0323 - val_loss: 0.0338
Epoch 1451/2000
1/1 [=

1/1 [==============================] - 0s 110ms/step - gen_loss: 0.5456 - disc_loss: 0.1130 - rmse: 0.0321 - val_loss: 0.0333
Epoch 1502/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.5453 - disc_loss: 0.1158 - rmse: 0.0322 - val_loss: 0.0339
Epoch 1503/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.5837 - disc_loss: 0.1105 - rmse: 0.0314 - val_loss: 0.0341
Epoch 1504/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 0.5843 - disc_loss: 0.1123 - rmse: 0.0321 - val_loss: 0.0337
Epoch 1505/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.4675 - disc_loss: 0.1142 - rmse: 0.0318 - val_loss: 0.0322
Epoch 1506/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 0.5165 - disc_loss: 0.1154 - rmse: 0.0285 - val_loss: 0.0317
Epoch 1507/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 0.5393 - disc_loss: 0.1099 - rmse: 0.0350 - val_loss: 0.0325
Epoch 1508/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 31ms/step - loss: 0.0323


0.03234165534377098

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 13)
120
x.shape = (105120, 13)
x_reshape.shape = (876, 120, 13)
159120
y_true.shape= (105120, 13)
159120


(1, 120, 13)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 13) (120, 13)
(105192, 13)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 13)
120
n= 105120
(35040, 9)
(35040, 9)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소        탁도     총유기탄소       총질소  \
0      0.130841  0.342105  0.545455  0.585185  0.002067  0.107692  0.412214   
1      0.124611  0.342105  0.545455  0.585185  0.001722  0.107692  0.430695   
2      0.127726  0.342105  0.545455  0.585185  0.001722  0.107692  0.441744   
3      0.127726  0.342105  0.545455  0.585185  0.002067  0.107692  0.429088   
4      0.127726  0.342105  0.545455  0.585185  0.002067  0.107692  0.423262   
...         ...       ...       ...       ...       ...       ...       ...   
35059  0.102804  0.236842  0.468531  0.718519  0.003789  0.169231  0.275613   
35060  0.099688  0.236842  0.468531  0.718519  0.003789  0.153846  0.277220   
35061  0.096573  0.236842  0.468531  0.718519  0.003789  0.153846  0.279630   
35062  0.096573  0.236842  0.468531  0.718519  0.003789  0.153846  0.276617   
35063  0.093458  0.236842  0.468531  0.718519  0.003445  0.153846  0.273001   

             총인    클로

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

(35040, 13) (35040, 13)
(35040, 9) (35040, 9)
(35040, 31) (35040, 31)


In [53]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [54]:
MAX_EPOCHS = 150
target_col = '클로로필-a'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,5,6,7,8]
out_num_features = len(out_features)
out_num_features

1

**예측결과 class**

In [55]:
cond = {
    "총인":[
        [-1000,0.02],
        [0.02,0.04],
        [0.04,0.10],
        [0.10,0.20],
        [0.20,0.30],
        [0.30,0.50],
        [0.50,1000],
    ],
    "총질소":[
        [-1000,0.2],
        [0.2,0.3],
        [0.3,0.4],
        [0.4,0.6],
        [0.6,1.0],
        [1.0,1.5],
        [1.5,1000],
    ],
    "수소이온농도":[
        [6.5 ,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.5,8.5],
        [6.0,8.5],
        [6.0,8.5],
        [-1000,1000],
    ],
    "총유기탄소":[
        [-1000 ,2],
        [2,3],
        [3,4],
        [4,5],
        [5,6],
        [6,8],
        [8,1000],
    ],
    "용존산소":[
        [7.5 ,1000],
        [5,7.5],
        [5,7.5],
        [5,7.5],
        [2,5],
        [2,5],
        [-1000,2],
    ],
    "클로로필-a":[
        [-1000 ,5],
        [5,9],
        [9,14],
        [14,20],
        [20,42],
        [42,70],
        [70,1000],
    ],
    
}

In [56]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [57]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features,
#                          out_features = out_features
                        )

In [58]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.130841,0.342105,0.545455,0.585185,0.002067,0.107692,0.412214,0.015190,0.035827,0.500000,...,0.040226,0.127726,0.421053,0.405594,0.725926,0.002067,0.092308,0.247135,0.021972,0.017599
1,0.124611,0.342105,0.545455,0.585185,0.001722,0.107692,0.430695,0.007595,0.033941,0.629410,...,0.045883,0.121495,0.421053,0.405594,0.725926,0.002067,0.092308,0.235540,0.018907,0.016970
2,0.127726,0.342105,0.545455,0.585185,0.001722,0.107692,0.441744,0.007595,0.033941,0.750000,...,0.042740,0.112150,0.421053,0.412587,0.711111,0.002067,0.092308,0.239537,0.029559,0.016970
3,0.127726,0.342105,0.545455,0.585185,0.002067,0.107692,0.429088,0.005063,0.033941,0.853553,...,0.033941,0.102804,0.394737,0.412587,0.711111,0.002067,0.092308,0.244445,0.024257,0.017599
4,0.127726,0.342105,0.545455,0.585185,0.002067,0.107692,0.423262,0.007595,0.033941,0.933013,...,0.043716,0.096573,0.394737,0.405594,0.711111,0.002067,0.092308,0.245291,0.025318,0.016342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.507788,0.184211,0.468531,0.474074,0.007578,0.169231,0.345520,0.022785,0.040226,0.017037,...,0.125079,0.482866,0.236842,0.251748,0.503704,0.003789,0.138462,0.174769,0.055696,0.023391
24524,0.504673,0.157895,0.461538,0.459259,0.007234,0.169231,0.351748,0.020253,0.037712,0.066987,...,0.124450,0.473520,0.210526,0.251748,0.481481,0.003445,0.138462,0.169546,0.058228,0.023712
24525,0.504673,0.157895,0.461538,0.459259,0.007234,0.169231,0.352551,0.022785,0.037712,0.146447,...,0.124450,0.457944,0.184211,0.258741,0.466667,0.002756,0.138462,0.168140,0.058228,0.044095
24526,0.504673,0.157895,0.461538,0.459259,0.007234,0.169231,0.353154,0.022785,0.037712,0.250000,...,0.123821,0.445483,0.184211,0.258741,0.444444,0.002411,0.138462,0.169546,0.055696,0.027469


In [59]:
it = iter(wdg)

In [60]:
x,y = next(it)
x.shape, y.shape

((128, 168, 31), (128, 72, 1))

In [61]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [62]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [63]:
def hour_to_day_mean(array):
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [65]:
!pip list 

Package                Version
---------------------- ---------
absl-py                0.11.0
argon2-cffi            20.1.0
astunparse             1.6.3
async-generator        1.10
attrs                  20.3.0
backcall               0.2.0
bleach                 3.2.1
blessings              1.7
cachetools             4.1.1
certifi                2020.11.8
cffi                   1.14.4
chardet                3.0.4
cycler                 0.10.0
decorator              4.4.2
defusedxml             0.6.0
entrypoints            0.3
et-xmlfile             1.0.1
flatbuffers            1.12
gast                   0.3.3
google-auth            1.23.0
google-auth-oauthlib   0.4.2
google-pasta           0.2.0
gpustat                0.6.0
grpcio                 1.32.0
h5py                   2.10.0
idna                   2.10
importlib-metadata     2.0.0
ipykernel              5.4.2
ipympl                 0.6.2
ipython                7.19.0
ipython-genutils       0.2.0
ipywidgets             7.6.2
jd

In [66]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None):
    
  if example is not None :
    inputs, labels = windows
  else :
    inputs, labels = self.example
 

  plot_out_col_index = self.column_indices[plot_out_col]

  if self.label_columns:
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_out_col_index = plot_out_col_index
    
    
  if model is None:
    return
    
  print("column : " + df_all.columns[plot_col])
    
  mae = 0
  mse = 0
  rmse = 0
  mape = 0
#   sk_mae = 0
#   sk_mse = 0
#   sk_rmse = 0

  pred_arr = []
  label_arr = []

  print(inputs.shape)
  print(len(inputs))


  for n in range(len(inputs)):
    predictions = model(inputs)
    pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
#     pred_temp = pred_temp * train_std[plot_col] + train_mean[plot_col]
    label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    pred_arr.append(pred_temp[2])
    label_arr.append(label_temp[2])
    
#     sk_mae = sk_mae + mean_absolute_error(label_temp, pred_temp)
#     print(aa)
  
#     error = label_temp[2] - pred_temp[2]
    
#     mae = mae + np.absolute(error)
#     mse = mse + error**2
    

    
  mae = mean_absolute_error(label_arr, pred_arr)
  mse = mean_squared_error(label_arr, pred_arr)
  rmse = np.sqrt(mse)
  mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100


#   mae = mae / len(inputs)
#   mse = mse / len(inputs)
  
#   rmse = np.sqrt(mse)
    
#   print("mae:")  
#   print(mae)
    
#   print("mse:")  
#   print(mse.numpy())
    
#   print("rmse")  
#   print(rmse)
    

  print("mae:")  
  print(mae)
    
  print("mse:")  
  print(mse)
    
  print("rmse")  
  print(rmse)
    
  print("mape")  
  print(mape)
    
    

WindowGenerator.compa = compa

In [67]:
train_df.shape

(24528, 31)

In [68]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [69]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [70]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [71]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [72]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [73]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


100/100 [==============================] - 8s 81ms/step - loss: 3.7033e-04 - mean_absolute_error: 0.0124
val performance = [0.00037133871228434145, 0.012426545843482018]
test performance =  [0.0009238390484824777, 0.01728126034140587]


In [74]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Linear**

In [75]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/150
1/1 [==============================] - 0s 443ms/step - loss: 0.0081 - mean_absolute_error: 0.0678 - val_loss: 0.0027 - val_mean_absolute_error: 0.0402
Epoch 2/150
1/1 [==============================] - 0s 217ms/step - loss: 0.0072 - mean_absolute_error: 0.0549 - val_loss: 0.0023 - val_mean_absolute_error: 0.0362
Epoch 3/150
1/1 [==============================] - 0s 265ms/step - loss: 0.0034 - mean_absolute_error: 0.0385 - val_loss: 0.0015 - val_mean_absolute_error: 0.0263
Epoch 4/150
1/1 [==============================] - 0s 232ms/step - loss: 0.0034 - mean_absolute_error: 0.0364 - val_loss: 0.0018 - val_mean_absolute_error: 0.0296
Epoch 5/150
1/1 [==============================] - 0s 229ms/step - loss: 0.0035 - mean_absolute_error: 0.0347 - val_loss: 0.0016 - val_mean_absolute_error: 0.0298
Epoch 6/150
1/1 [==============================] - 0s 234ms/step - loss: 0.0022 - mean_absolute_error: 0.0304 - val_loss: 0.0011 - val_mean_absolute_error: 0.0266
Epoch 7/150
1/1 [=====

Epoch 51/150
1/1 [==============================] - 0s 213ms/step - loss: 0.0019 - mean_absolute_error: 0.0297 - val_loss: 0.0011 - val_mean_absolute_error: 0.0259
Epoch 52/150
1/1 [==============================] - 0s 243ms/step - loss: 0.0022 - mean_absolute_error: 0.0304 - val_loss: 0.0010 - val_mean_absolute_error: 0.0258
Epoch 53/150
1/1 [==============================] - 0s 195ms/step - loss: 0.0020 - mean_absolute_error: 0.0291 - val_loss: 0.0012 - val_mean_absolute_error: 0.0277
Epoch 54/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0020 - mean_absolute_error: 0.0308 - val_loss: 9.9623e-04 - val_mean_absolute_error: 0.0251
Epoch 55/150
1/1 [==============================] - 0s 239ms/step - loss: 0.0013 - mean_absolute_error: 0.0252 - val_loss: 8.9879e-04 - val_mean_absolute_error: 0.0247
Epoch 56/150
1/1 [==============================] - 0s 266ms/step - loss: 0.0026 - mean_absolute_error: 0.0342 - val_loss: 0.0011 - val_mean_absolute_error: 0.0266
Epoch 57

1/1 [==============================] - 0s 215ms/step - loss: 0.0018 - mean_absolute_error: 0.0279 - val_loss: 8.7623e-04 - val_mean_absolute_error: 0.0234
Epoch 101/150
1/1 [==============================] - 0s 226ms/step - loss: 0.0020 - mean_absolute_error: 0.0288 - val_loss: 0.0011 - val_mean_absolute_error: 0.0251
Epoch 102/150
1/1 [==============================] - 0s 208ms/step - loss: 0.0023 - mean_absolute_error: 0.0307 - val_loss: 0.0012 - val_mean_absolute_error: 0.0266
Epoch 103/150
1/1 [==============================] - 0s 235ms/step - loss: 0.0014 - mean_absolute_error: 0.0267 - val_loss: 9.8565e-04 - val_mean_absolute_error: 0.0245
Epoch 104/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0024 - mean_absolute_error: 0.0293 - val_loss: 8.6885e-04 - val_mean_absolute_error: 0.0240
Epoch 105/150
1/1 [==============================] - 0s 258ms/step - loss: 0.0022 - mean_absolute_error: 0.0277 - val_loss: 0.0011 - val_mean_absolute_error: 0.0260
Epoch 106/15

1/1 [==============================] - 0s 206ms/step - loss: 0.0016 - mean_absolute_error: 0.0264 - val_loss: 0.0011 - val_mean_absolute_error: 0.0257
Epoch 150/150
100/100 [==============================] - 8s 80ms/step - loss: 9.7269e-04 - mean_absolute_error: 0.0243
val performance = [0.0009726886637508869, 0.024285895749926567]
test performance =  [0.0013324965257197618, 0.025327326729893684]


In [76]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [78]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Elman_RNN**

In [80]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

In [81]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [82]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [83]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 758ms/step - loss: 0.0122 - mean_absolute_error: 0.0772 - val_loss: 0.0016 - val_mean_absolute_error: 0.0273
Epoch 2/150
1/1 [==============================] - 0s 267ms/step - loss: 0.0044 - mean_absolute_error: 0.0403 - val_loss: 0.0017 - val_mean_absolute_error: 0.0372
Epoch 3/150
1/1 [==============================] - 0s 280ms/step - loss: 0.0031 - mean_absolute_error: 0.0413 - val_loss: 0.0031 - val_mean_absolute_error: 0.0511
Epoch 4/150
1/1 [==============================] - 0s 263ms/step - loss: 0.0043 - mean_absolute_error: 0.0538 - val_loss: 0.0014 - val_mean_absolute_error: 0.0309
Epoch 5/150
1/1 [==============================] - 0s 269ms/step - loss: 0.0020 - mean_absolute_error: 0.0367 - val_loss: 0.0013 - val_mean_absolute_error: 0.0263
Epoch 6/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0019 - mean_absolute_error: 0.0278 - val_loss: 0.0011 - val_mean_absolute_error: 0.0245
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 271ms/step - loss: 0.0020 - mean_absolute_error: 0.0255 - val_loss: 8.7505e-04 - val_mean_absolute_error: 0.0212
Epoch 51/150
1/1 [==============================] - 0s 252ms/step - loss: 0.0011 - mean_absolute_error: 0.0219 - val_loss: 8.2592e-04 - val_mean_absolute_error: 0.0215
Epoch 52/150
1/1 [==============================] - 0s 275ms/step - loss: 9.2490e-04 - mean_absolute_error: 0.0231 - val_loss: 7.3962e-04 - val_mean_absolute_error: 0.0213
Epoch 53/150
1/1 [==============================] - 0s 273ms/step - loss: 8.2894e-04 - mean_absolute_error: 0.0214 - val_loss: 0.0010 - val_mean_absolute_error: 0.0225
Epoch 54/150
1/1 [==============================] - 0s 273ms/step - loss: 7.8676e-04 - mean_absolute_error: 0.0205 - val_loss: 0.0010 - val_mean_absolute_error: 0.0218
Epoch 55/150
1/1 [==============================] - 0s 300ms/step - loss: 0.0011 - mean_absolute_error: 0.0224 - val_loss: 0.0010 - val_mean_absolute_error: 0.0211
Epoch

1/1 [==============================] - 0s 268ms/step - loss: 0.0010 - mean_absolute_error: 0.0216 - val_loss: 7.0062e-04 - val_mean_absolute_error: 0.0189
Epoch 99/150
1/1 [==============================] - 0s 270ms/step - loss: 6.4962e-04 - mean_absolute_error: 0.0174 - val_loss: 6.5974e-04 - val_mean_absolute_error: 0.0183
Epoch 100/150
1/1 [==============================] - 0s 279ms/step - loss: 8.7151e-04 - mean_absolute_error: 0.0190 - val_loss: 7.7466e-04 - val_mean_absolute_error: 0.0198
Epoch 101/150
1/1 [==============================] - 0s 276ms/step - loss: 0.0010 - mean_absolute_error: 0.0201 - val_loss: 5.2982e-04 - val_mean_absolute_error: 0.0188
Epoch 102/150
1/1 [==============================] - 0s 241ms/step - loss: 7.8959e-04 - mean_absolute_error: 0.0221 - val_loss: 5.3259e-04 - val_mean_absolute_error: 0.0173
Epoch 103/150
1/1 [==============================] - 0s 266ms/step - loss: 0.0013 - mean_absolute_error: 0.0234 - val_loss: 5.2834e-04 - val_mean_absolute_err

Epoch 146/150
1/1 [==============================] - 0s 245ms/step - loss: 0.0012 - mean_absolute_error: 0.0225 - val_loss: 4.8279e-04 - val_mean_absolute_error: 0.0166
Epoch 147/150
1/1 [==============================] - 0s 271ms/step - loss: 7.0009e-04 - mean_absolute_error: 0.0172 - val_loss: 5.6779e-04 - val_mean_absolute_error: 0.0180
Epoch 148/150
1/1 [==============================] - 0s 288ms/step - loss: 7.1727e-04 - mean_absolute_error: 0.0192 - val_loss: 5.6522e-04 - val_mean_absolute_error: 0.0164
Epoch 149/150
1/1 [==============================] - 0s 269ms/step - loss: 9.1608e-04 - mean_absolute_error: 0.0207 - val_loss: 5.4845e-04 - val_mean_absolute_error: 0.0169
Epoch 150/150
100/100 [==============================] - 9s 85ms/step - loss: 7.2847e-04 - mean_absolute_error: 0.0178
val performance = [0.0005732813151553273, 0.01669054850935936]
test performance =  [0.0007284749881364405, 0.01777481846511364]


In [84]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [86]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [87]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 5s 5s/step - loss: 0.0048 - mean_absolute_error: 0.0541 - val_loss: 0.0035 - val_mean_absolute_error: 0.0489
Epoch 2/150
1/1 [==============================] - 1s 922ms/step - loss: 0.0056 - mean_absolute_error: 0.0560 - val_loss: 0.0033 - val_mean_absolute_error: 0.0466
Epoch 3/150
1/1 [==============================] - 1s 922ms/step - loss: 0.0065 - mean_absolute_error: 0.0565 - val_loss: 0.0016 - val_mean_absolute_error: 0.0275
Epoch 4/150
1/1 [==============================] - 1s 940ms/step - loss: 0.0035 - mean_absolute_error: 0.0382 - val_loss: 0.0024 - val_mean_absolute_error: 0.0450
Epoch 5/150
1/1 [==============================] - 1s 935ms/step - loss: 0.0036 - mean_absolute_error: 0.0478 - val_loss: 0.0017 - val_mean_absolute_error: 0.0374
Epoch 6/150
1/1 [==============================] - 1s 920ms/step - loss: 0.0028 - mean_absolute_error: 0.0406 - val_loss: 0.0013 - val_mean_absolute_error: 0.0275
Epoch 7/150
1/1 [========

Epoch 51/150
1/1 [==============================] - 1s 958ms/step - loss: 8.9457e-04 - mean_absolute_error: 0.0214 - val_loss: 0.0010 - val_mean_absolute_error: 0.0254
Epoch 52/150
1/1 [==============================] - 1s 931ms/step - loss: 0.0012 - mean_absolute_error: 0.0242 - val_loss: 6.3897e-04 - val_mean_absolute_error: 0.0191
Epoch 53/150
1/1 [==============================] - 1s 900ms/step - loss: 8.4359e-04 - mean_absolute_error: 0.0208 - val_loss: 8.1345e-04 - val_mean_absolute_error: 0.0203
Epoch 54/150
1/1 [==============================] - 1s 922ms/step - loss: 0.0012 - mean_absolute_error: 0.0231 - val_loss: 8.4944e-04 - val_mean_absolute_error: 0.0206
Epoch 55/150
1/1 [==============================] - 1s 945ms/step - loss: 8.9525e-04 - mean_absolute_error: 0.0198 - val_loss: 7.8629e-04 - val_mean_absolute_error: 0.0217
Epoch 56/150
1/1 [==============================] - 1s 940ms/step - loss: 0.0017 - mean_absolute_error: 0.0238 - val_loss: 0.0012 - val_mean_absolute_er

1/1 [==============================] - 1s 892ms/step - loss: 0.0021 - mean_absolute_error: 0.0347 - val_loss: 7.3560e-04 - val_mean_absolute_error: 0.0228
Epoch 100/150
1/1 [==============================] - 1s 905ms/step - loss: 7.9177e-04 - mean_absolute_error: 0.0225 - val_loss: 4.5698e-04 - val_mean_absolute_error: 0.0149
Epoch 101/150
1/1 [==============================] - 1s 898ms/step - loss: 9.6487e-04 - mean_absolute_error: 0.0194 - val_loss: 9.3239e-04 - val_mean_absolute_error: 0.0227
Epoch 102/150
1/1 [==============================] - 1s 915ms/step - loss: 0.0015 - mean_absolute_error: 0.0254 - val_loss: 8.5502e-04 - val_mean_absolute_error: 0.0208
Epoch 103/150
1/1 [==============================] - 1s 924ms/step - loss: 0.0012 - mean_absolute_error: 0.0240 - val_loss: 4.5174e-04 - val_mean_absolute_error: 0.0168
Epoch 104/150
1/1 [==============================] - 1s 941ms/step - loss: 7.2388e-04 - mean_absolute_error: 0.0176 - val_loss: 8.9185e-04 - val_mean_absolute_er

Epoch 147/150
1/1 [==============================] - 1s 928ms/step - loss: 0.0010 - mean_absolute_error: 0.0224 - val_loss: 5.3500e-04 - val_mean_absolute_error: 0.0178
Epoch 148/150
1/1 [==============================] - 1s 915ms/step - loss: 9.5307e-04 - mean_absolute_error: 0.0200 - val_loss: 5.5293e-04 - val_mean_absolute_error: 0.0171
Epoch 149/150
1/1 [==============================] - 1s 897ms/step - loss: 7.9401e-04 - mean_absolute_error: 0.0185 - val_loss: 7.5793e-04 - val_mean_absolute_error: 0.0189
Epoch 150/150
100/100 [==============================] - 22s 217ms/step - loss: 9.8179e-04 - mean_absolute_error: 0.0195
val performance = [0.0006664047250524163, 0.017830414697527885]
test performance =  [0.0009817886166274548, 0.019536064937710762]


In [88]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
multi_window.plot2(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**LSTM**

In [90]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [92]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 0.0061 - mean_absolute_error: 0.0619 - val_loss: 0.0035 - val_mean_absolute_error: 0.0462
Epoch 2/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0047 - mean_absolute_error: 0.0518 - val_loss: 0.0021 - val_mean_absolute_error: 0.0314
Epoch 3/150
1/1 [==============================] - 0s 253ms/step - loss: 0.0047 - mean_absolute_error: 0.0453 - val_loss: 0.0011 - val_mean_absolute_error: 0.0269
Epoch 4/150
1/1 [==============================] - 0s 293ms/step - loss: 0.0022 - mean_absolute_error: 0.0306 - val_loss: 0.0031 - val_mean_absolute_error: 0.0512
Epoch 5/150
1/1 [==============================] - 0s 247ms/step - loss: 0.0034 - mean_absolute_error: 0.0519 - val_loss: 0.0025 - val_mean_absolute_error: 0.0460
Epoch 6/150
1/1 [==============================] - 0s 251ms/step - loss: 0.0047 - mean_absolute_error: 0.0497 - val_loss: 0.0013 - val_mean_absolute_error: 0.0315
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 264ms/step - loss: 0.0013 - mean_absolute_error: 0.0232 - val_loss: 8.2430e-04 - val_mean_absolute_error: 0.0225
Epoch 51/150
1/1 [==============================] - 0s 292ms/step - loss: 0.0014 - mean_absolute_error: 0.0259 - val_loss: 9.6244e-04 - val_mean_absolute_error: 0.0232
Epoch 52/150
1/1 [==============================] - 0s 287ms/step - loss: 0.0020 - mean_absolute_error: 0.0296 - val_loss: 6.8795e-04 - val_mean_absolute_error: 0.0187
Epoch 53/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0011 - mean_absolute_error: 0.0218 - val_loss: 0.0010 - val_mean_absolute_error: 0.0227
Epoch 54/150
1/1 [==============================] - 0s 268ms/step - loss: 0.0012 - mean_absolute_error: 0.0242 - val_loss: 0.0014 - val_mean_absolute_error: 0.0269
Epoch 55/150
1/1 [==============================] - 0s 261ms/step - loss: 0.0019 - mean_absolute_error: 0.0280 - val_loss: 7.5986e-04 - val_mean_absolute_error: 0.0191
Epoch 56/150


1/1 [==============================] - 0s 269ms/step - loss: 7.0901e-04 - mean_absolute_error: 0.0198 - val_loss: 6.4671e-04 - val_mean_absolute_error: 0.0180
Epoch 99/150
1/1 [==============================] - 0s 279ms/step - loss: 5.2861e-04 - mean_absolute_error: 0.0167 - val_loss: 5.6702e-04 - val_mean_absolute_error: 0.0168
Epoch 100/150
1/1 [==============================] - 0s 266ms/step - loss: 0.0012 - mean_absolute_error: 0.0222 - val_loss: 7.7159e-04 - val_mean_absolute_error: 0.0191
Epoch 101/150
1/1 [==============================] - 0s 278ms/step - loss: 6.5608e-04 - mean_absolute_error: 0.0184 - val_loss: 6.1461e-04 - val_mean_absolute_error: 0.0185
Epoch 102/150
1/1 [==============================] - 0s 283ms/step - loss: 9.4232e-04 - mean_absolute_error: 0.0202 - val_loss: 0.0012 - val_mean_absolute_error: 0.0242
Epoch 103/150
1/1 [==============================] - 0s 252ms/step - loss: 7.2307e-04 - mean_absolute_error: 0.0200 - val_loss: 6.3935e-04 - val_mean_absolute

Epoch 146/150
1/1 [==============================] - 0s 284ms/step - loss: 0.0014 - mean_absolute_error: 0.0197 - val_loss: 6.7396e-04 - val_mean_absolute_error: 0.0175
Epoch 147/150
1/1 [==============================] - 0s 243ms/step - loss: 7.6256e-04 - mean_absolute_error: 0.0187 - val_loss: 6.2869e-04 - val_mean_absolute_error: 0.0166
Epoch 148/150
1/1 [==============================] - 0s 283ms/step - loss: 0.0011 - mean_absolute_error: 0.0197 - val_loss: 8.6430e-04 - val_mean_absolute_error: 0.0192
Epoch 149/150
1/1 [==============================] - 0s 252ms/step - loss: 7.7103e-04 - mean_absolute_error: 0.0176 - val_loss: 6.9583e-04 - val_mean_absolute_error: 0.0184
Epoch 150/150
1/1 [==============================] - 0s 147ms/step - loss: 7.7770e-04 - mean_absolute_error: 0.0196
val performance = [0.0007776996353641152, 0.019578510895371437]
test performance =  [0.001084161689504981, 0.0214050505310297]


In [93]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**CNN model**

In [95]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [96]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 714ms/step - loss: 0.0056 - mean_absolute_error: 0.0568 - val_loss: 0.0013 - val_mean_absolute_error: 0.0248
Epoch 2/150
1/1 [==============================] - 0s 231ms/step - loss: 0.0020 - mean_absolute_error: 0.0299 - val_loss: 0.0036 - val_mean_absolute_error: 0.0553
Epoch 3/150
1/1 [==============================] - 0s 227ms/step - loss: 0.0054 - mean_absolute_error: 0.0619 - val_loss: 0.0018 - val_mean_absolute_error: 0.0380
Epoch 4/150
1/1 [==============================] - 0s 248ms/step - loss: 0.0035 - mean_absolute_error: 0.0450 - val_loss: 0.0011 - val_mean_absolute_error: 0.0265
Epoch 5/150
1/1 [==============================] - 0s 227ms/step - loss: 0.0021 - mean_absolute_error: 0.0311 - val_loss: 0.0010 - val_mean_absolute_error: 0.0224
Epoch 6/150
1/1 [==============================] - 0s 249ms/step - loss: 0.0034 - mean_absolute_error: 0.0327 - val_loss: 0.0012 - val_mean_absolute_error: 0.0259
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 224ms/step - loss: 0.0012 - mean_absolute_error: 0.0218 - val_loss: 7.9641e-04 - val_mean_absolute_error: 0.0212
Epoch 51/150
1/1 [==============================] - 0s 224ms/step - loss: 8.5742e-04 - mean_absolute_error: 0.0199 - val_loss: 7.2861e-04 - val_mean_absolute_error: 0.0201
Epoch 52/150
1/1 [==============================] - 0s 278ms/step - loss: 6.3467e-04 - mean_absolute_error: 0.0192 - val_loss: 7.7330e-04 - val_mean_absolute_error: 0.0189
Epoch 53/150
1/1 [==============================] - 0s 234ms/step - loss: 0.0013 - mean_absolute_error: 0.0202 - val_loss: 4.6829e-04 - val_mean_absolute_error: 0.0157
Epoch 54/150
1/1 [==============================] - 0s 245ms/step - loss: 0.0010 - mean_absolute_error: 0.0193 - val_loss: 6.9523e-04 - val_mean_absolute_error: 0.0193
Epoch 55/150
1/1 [==============================] - 0s 221ms/step - loss: 9.8062e-04 - mean_absolute_error: 0.0200 - val_loss: 7.0008e-04 - val_mean_absolute_error: 

1/1 [==============================] - 0s 261ms/step - loss: 5.4989e-04 - mean_absolute_error: 0.0171 - val_loss: 6.1639e-04 - val_mean_absolute_error: 0.0174
Epoch 99/150
1/1 [==============================] - 0s 242ms/step - loss: 7.9650e-04 - mean_absolute_error: 0.0174 - val_loss: 6.4530e-04 - val_mean_absolute_error: 0.0199
Epoch 100/150
1/1 [==============================] - 0s 243ms/step - loss: 7.7109e-04 - mean_absolute_error: 0.0203 - val_loss: 4.9912e-04 - val_mean_absolute_error: 0.0176
Epoch 101/150
1/1 [==============================] - 0s 252ms/step - loss: 7.0497e-04 - mean_absolute_error: 0.0198 - val_loss: 5.3386e-04 - val_mean_absolute_error: 0.0177
Epoch 102/150
1/1 [==============================] - 0s 226ms/step - loss: 8.6572e-04 - mean_absolute_error: 0.0189 - val_loss: 5.4389e-04 - val_mean_absolute_error: 0.0163
Epoch 103/150
1/1 [==============================] - 0s 249ms/step - loss: 0.0010 - mean_absolute_error: 0.0191 - val_loss: 8.4173e-04 - val_mean_abso

Epoch 146/150
1/1 [==============================] - 0s 246ms/step - loss: 6.4273e-04 - mean_absolute_error: 0.0170 - val_loss: 4.7346e-04 - val_mean_absolute_error: 0.0148
Epoch 147/150
1/1 [==============================] - 0s 217ms/step - loss: 0.0011 - mean_absolute_error: 0.0175 - val_loss: 4.9219e-04 - val_mean_absolute_error: 0.0156
Epoch 148/150
1/1 [==============================] - 0s 206ms/step - loss: 8.9768e-04 - mean_absolute_error: 0.0172 - val_loss: 5.3107e-04 - val_mean_absolute_error: 0.0159
Epoch 149/150
1/1 [==============================] - 0s 234ms/step - loss: 7.4775e-04 - mean_absolute_error: 0.0158 - val_loss: 4.0084e-04 - val_mean_absolute_error: 0.0141
Epoch 150/150
1/1 [==============================] - 0s 126ms/step - loss: 4.8682e-04 - mean_absolute_error: 0.0147
val performance = [0.0004868245159741491, 0.014729168266057968]
test performance =  [0.0003752675256691873, 0.013812207616865635]


In [97]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 126ms/step - loss: 4.0545e-04 - mean_absolute_error: 0.0135
val performance = [0.0004054520686622709, 0.013453975319862366]
test performance =  [0.000641773862298578, 0.014752326533198357]


In [98]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**performance**

In [100]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 